In [3]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Add parent directory to path to import local modules
sys.path.insert(0, str(Path.cwd().parent))

from src.behavior_import.import_reward_bandit_data import *

In [9]:
data = import_reward_bandit_data("../data/cohort-2/rawdata")

[INFO] Built 6 subject(s), 36 session(s).


In [13]:
print(data.keys())

dict_keys(['MY_05_R', 'MY_05_L', 'MY_05_N', 'MY_04_N', 'MY_04_R', 'MY_04_L'])


In [14]:
print(data['MY_05_R'].keys())

dict_keys(['ses-06_2026-01-13', 'ses-05_2026-01-13', 'ses-04_2026-01-12', 'ses-03_2026-01-12', 'ses-02_2026-01-11', 'ses-01_2026-01-11'])
